# Detect the colors of each address in a folder


Ideally this should be run in colab or a machine with good reasources.

In [ ]:
# In colab one can download all the images to a file then upload to drive and get them from from 

# google.colab import drive
# drive.mount('/content/drive') 

In [1]:
# imports
import os
from tabulate import tabulate

## Load images from source directory:

In [28]:
#If using Colab, make sure to input the right path to your dataset
dataset = "borgundvegen1"
Source = os.listdir(f'data/buildings/{dataset}')
print(Source[0])

# Other options
options = os.listdir("data/buildings")[:4]
print(f'Other options: {options}')

Borgundvegen168B_6008_AALESUND_62.47102613101382_6.20102655645523_East.png
Other options: ['aspoey', 'bjoerkavaagdalen', 'borgundvegen1', 'borgundvegen2']


Making a class and helpful functions for a better structure.

In [3]:
class Building:
    def __init__(self, addr, pC, city_, lat, long, dir, type):
        self.address = addr
        self.postalCode = pC
        self.city = city_
        self.latitude = lat
        self.longitude = long
        self.filetype = type
        self.directions = [dir]
        self.filenames = []
        self.predictions = {
                                'North':("empty", 0),
                                'East': ("empty", 0),
                                'West': ("empty", 0),
                                'South': ("empty", 0)
                            }
        self.predicted = "None"
    
    def getTableItems(self):
        return [
                    self.address,
                    f"{self.postalCode}, {self.city}",
                    f"{self.latitude} N",
                    f"{self.longitude} E", 
                    len(self.directions),
                    self.predicted, 
                    self.predictions["North"],
                    self.predictions["East"],
                    self.predictions["West"],
                    self.predictions["South"],
                ]
    
    def addPrediction(self, filename: str, prediction:tuple):
        direction = filename.split('_')[-1].split('.')[0]
        self.predictions[direction] = prediction
    
    def MakePrediction(self):
        votes = {}
        for key, predictionTuple in self.predictions.items():
            class_, conf_ = predictionTuple
            # Choosing the one with the highest combined confidence
            if class_ == "empty":
                continue
            elif class_ not in votes:
                votes[class_] = float(conf_)
            else: 
                votes[class_] += float(conf_)
        if len(votes) == 0:
            self.predicted = ("empty", 0)
        else:
            self.predicted = max(votes, key=votes.get)
            
    def getDumpformat(self):
        return [
                    self.postalCode,
                    self.city,
                    self.latitude,
                    self.longitude,
                    self.predicted,
                    self.predictions["North"],
                    self.predictions["East"],
                    self.predictions["West"],
                    self.predictions["South"],
                ]

In [4]:
def readInSources(Source):
    Buildings = dict()
    for filename in Source:
        filename_splitted = filename.split('_')
        
        # If the filename is not in the correct format, skip it. This means it is a building plot or something like that
        if filename_splitted[0].isnumeric(): 
            print(f"bad file encounterd: {filename}")
            print()
            continue
        # else continue 
        
        address, postalCode, city, latitude, longitude, directionAndFiletype = filename_splitted
        direction, filetype = directionAndFiletype.split('.')
        
        if address in Buildings:
            Buildings[address].directions.append(direction)
        else:
            Buildings[address] = Building(address, postalCode, city, latitude, longitude, direction, filetype)
        Buildings[address].filenames.append(f"data/buildings/{dataset}/{filename}")
    return Buildings
            

In [5]:
def PrintStatistics(Buildings):
    header = ['Address', 'Postal address', 'Lat', 'Long', 'Directions', 'Predicted','North', 'East', 'West', 'South']

    tableContent = []
    for _, building in Buildings.items():
        tableContent.append(building.getTableItems())
    print(tabulate(tableContent, headers=header))

In [8]:
# Note that the printed output should be empty, as we have not run the model yet.

SourceBuildings = readInSources(Source)
print("Table before runnning the model")
PrintStatistics(SourceBuildings)

Table before runnning the model
Address           Postal address    Lat                   Long                    Directions  Predicted    North         East          West          South
----------------  ----------------  --------------------  --------------------  ------------  -----------  ------------  ------------  ------------  ------------
Borgundvegen168B  6008, AALESUND    62.47102613101382 N   6.20102655645523 E               4  None         ('empty', 0)  ('empty', 0)  ('empty', 0)  ('empty', 0)
Borgundvegen168C  6008, AALESUND    62.47088407022519 N   6.201037339998814 E              4  None         ('empty', 0)  ('empty', 0)  ('empty', 0)  ('empty', 0)
Borgundvegen169   6008, AALESUND    62.47150284793442 N   6.199806683465084 E              4  None         ('empty', 0)  ('empty', 0)  ('empty', 0)  ('empty', 0)
Borgundvegen170A  6008, AALESUND    62.47115199000118 N   6.201569186689616 E              4  None         ('empty', 0)  ('empty', 0)  ('empty', 0)  ('empty', 0)
Bor

## Loading the model

In [9]:
# More imports
from IPython import display
display.clear_output()
import ultralytics
ultralytics.checks()
from ultralytics import YOLO
from IPython.display import display, Image

Ultralytics YOLOv8.0.123  Python-3.10.1 torch-2.0.1+cpu CPU
Setup complete  (8 CPUs, 15.7 GB RAM, 161.8/475.7 GB disk)


In [10]:
pathToModel = "data/models/v12_larger.pt"

model = YOLO(pathToModel)

Helpful functions, (similar to the one in `demo.ipynb`)

In [11]:
def gradeResults(result):
    predictions = []
    
    imgWidth = result[0].orig_shape[0]
    imgHeight = result[0].orig_shape[1]
    
    for box in result[0].boxes:
        predictedClass = model.names[int(box.cls)]
        confidence = round(float(box.conf), 3)
        
        x, y, w, h = box.xywh[0]
        x_offset = imgWidth/2 - (x)
        y_offset = imgHeight/2 - (y)
        offset = x_offset**2 + y_offset**2
        predictions.append((predictedClass, confidence, offset))
    # Outputs list of guesses, confidence, and offset from center.
    return predictions

def chooseTheMiddleOne(predictions):
    if len(predictions) == 0:
        return "empty", 0
    # Sorts by offset from center
    predictions.sort(key=lambda x: x[2])
    
    minAcceptableOffset = 400
    if predictions[0][2] > minAcceptableOffset**2:
        return "bad", 0
    
    MiddleBoxClass, confidence = predictions[0][:-1]
    return (MiddleBoxClass, confidence)

Making the predictions

In [12]:
for building in SourceBuildings.values():
    for filename in building.filenames:
        ProcessedImage = model(filename)
        gradedPredictions = gradeResults(ProcessedImage)
        pred = chooseTheMiddleOne(gradedPredictions)
        building.addPrediction(filename, pred)
    building.MakePrediction()


image 1/1 c:\Users\elias\kystverket\building-colors\yolo_implementation\data\buildings\borgundvegen1\Borgundvegen168B_6008_AALESUND_62.47102613101382_6.20102655645523_East.png: 640x640 1 grey, 3 whites, 1338.2ms
Speed: 10.3ms preprocess, 1338.2ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\elias\kystverket\building-colors\yolo_implementation\data\buildings\borgundvegen1\Borgundvegen168B_6008_AALESUND_62.47102613101382_6.20102655645523_North.png: 640x640 2 greys, 1 red, 1273.3ms
Speed: 5.0ms preprocess, 1273.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\elias\kystverket\building-colors\yolo_implementation\data\buildings\borgundvegen1\Borgundvegen168B_6008_AALESUND_62.47102613101382_6.20102655645523_South.png: 640x640 1 red, 1243.1ms
Speed: 7.0ms preprocess, 1243.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\elias\kystverket\building-colors\yolo_implementation\d

In [13]:
PrintStatistics(SourceBuildings)

Address           Postal address    Lat                   Long                    Directions  Predicted    North              East               West               South
----------------  ----------------  --------------------  --------------------  ------------  -----------  -----------------  -----------------  -----------------  ----------------
Borgundvegen168B  6008, AALESUND    62.47102613101382 N   6.20102655645523 E               4  white        ('grey', 0.306)    ('white', 0.389)   ('brown', 0.263)   ('bad', 0)
Borgundvegen168C  6008, AALESUND    62.47088407022519 N   6.201037339998814 E              4  white        ('white', 0.414)   ('grey', 0.267)    ('white', 0.329)   ('brown', 0.255)
Borgundvegen169   6008, AALESUND    62.47150284793442 N   6.199806683465084 E              4  white        ('grey', 0.498)    ('white', 0.803)   ('blue', 0.754)    ('white', 0.801)
Borgundvegen170A  6008, AALESUND    62.47115199000118 N   6.201569186689616 E              4  white        ('whi

In [26]:
import json

def DumpResults(Buildings, filename):
    DumpDictionary = {}
    for addr in Buildings:
      DumpDictionary[addr] = Buildings[addr].getDumpformat()
    with open(f'{filename}_results.json', 'w') as fp:
        json.dump(DumpDictionary, fp)

Save the output to a .json file. To view the results in a nice format use the `load_json.ipynb` program.

In [27]:
path = "data/json/test"
DumpResults(SourceBuildings, path)